# TDT4173 - Project report for Group 62 (short notebook)

**We did the Kaggle competition named Sales Forecasting and are named Group 62 as Kaggle Group name.**

The names and student IDs for those on Group 62 are:
1. **Ruben Mustad, 480858**
2. **Aneeq Ahsan, 546865**
3. **Axel Luiggi-Gørrissen, 554086**

This notebook contains all the necessary information to create our two chosen submissions on Kaggle. Only the given dataset was used, and features was created using the information found in these datasets.

In [7]:
import numpy as np # version 1.21.0 was used
import pandas as pd # version 1.3.4 was used
import lightgbm as lgbm # version 3.2.1 was used
import warnings 
warnings.filterwarnings('ignore')
print(np.__version__)
print(pd.__version__)
print(lgbm.__version__)

1.21.0
1.3.4
3.2.1


In [2]:
def create_lag(train, test):
    # Create lag features for our model
    # We use lag of 60, 90, 180 and 365
    train['train'] = 1
    test['train'] = 0
    combined = pd.concat([train,test])
    lag = [60, 90, 180, 365]
    lag_cols = [f'lag_{lag}' for lag in lag]
    lag_colP = [f'lagPromo_{lag}' for lag in lag]
    lagPromo = [f'lag{lag}Promo' for lag in lag]
    for lags, lag_col in zip(lag,lag_cols): # create lag features
        combined[lag_col] = combined[['ts_id', 'Sales']].groupby('ts_id')['Sales'].shift(lags)
    for lags, lag_colPs in zip(lag,lag_colP): 
        combined[lag_colPs] = combined[['ts_id', 'isPromo']].groupby('ts_id')['isPromo'].shift(lags)
    train = combined[combined['train'] == 1]
    test = combined[combined['train'] == 0]
    train.drop(['train'], axis = 1, inplace = True)
    test.drop(['train'], axis = 1, inplace = True)
    def lag_creator(df): # create features with information about the lagged sale number and if it was promo or not
        df['lag60Promo'] = df['lag_60'].astype(str) + df['lagPromo_60'].astype(str)
        df['lag180Promo'] = df['lag_180'].astype(str) + df['lagPromo_180'].astype(str)
        df['lag90Promo'] = df['lag_90'].astype(str) + df['lagPromo_90'].astype(str)
        df['lag365Promo'] = df['lag_365'].astype(str) + df['lagPromo_365'].astype(str)
        return df
    train = lag_creator(train)
    test = lag_creator(test)
    remove = ['lagPromo_60', 'lagPromo_90', 'lagPromo_180', 'lagPromo_365']
    for i in remove:
        train.pop(i)
        test.pop(i)
    del(train['id'])
    del(test['Sales'])
    return train, test
def merge(df, sku_features, id_map):
    # Merge the given datasets
    df = pd.merge(df, id_map, how='left', on='ts_id')
    df = pd.merge(df, sku_features, how='left', on='SKU')
    return df
def time_features(df):
    # Add time features to our model
    df['day_of_month'] = df['Date'].dt.day
    df['day_of_week'] = df['Date'].dt.weekday
    df['month'] = df['Date'].dt.month
    df['year'] = df['Date'].dt.year
    df['week'] = df['Date'].dt.week
    return df
def promo_features(df):
    # Add promo features to most of the features in the given dataset
    df['ts_promo'] = df['ts_id'].astype(str) + df['isPromo'].astype(str)
    df['store_promo'] = df['Store'].astype(str) + df['isPromo'].astype(str)
    df['segment_promo'] = df['Segment'].astype(str) + df['isPromo'].astype(str)
    df['brand_promo'] = df['Brand'].astype(str) + df['isPromo'].astype(str)
    df['sku_promo'] = df['SKU'].astype(str) + df['isPromo'].astype(str)
    df['dom_promo'] = df['day_of_month'].astype(str) + df['isPromo'].astype(str)
    df['dow_promo'] = df['day_of_week'].astype(str) + df['isPromo'].astype(str)
    return df
def brand_fixer(df):
    # Fix the mistake in Baltik N0
    SKU_baltik0 = df['SKU'] == '_Baltika №0 Can 0,45'
    Brand_baltik0 = df['Brand'] == 'Baltika №3'
    SKU_index = SKU_baltik0[SKU_baltik0].index
    Brand_index = Brand_baltik0[Brand_baltik0].index
    
    baltik0 = list(set(SKU_index).intersection(Brand_index))
    df['Brand'][baltik0] = 'Baltika №0'
    return df
def lager_fix(train, test):
    # We found information about Firmennoe and Baltika Draught been lager
    for i in range(len(train)):
        if train['SKU'][i] == '_Firmennoe PET 1,42':
            train['Product'][i] = 'lager'
        elif train['SKU'][i] == '_Baltika Draught PET 0,47':
            train['Product'][i] = 'lager'
    for i in range(len(test)):
        if test['SKU'][i] == '_Firmennoe PET 1,42':
            test['Product'][i] = 'lager'
        elif test['SKU'][i] == '_Baltika Draught PET 0,47':
            test['Product'][i] = 'lager'
    return train, test
def remove2015(train):
    # Remove all of 2015
    train = train[train['year'] > 2015]
    return train

def remove_values(train, upper, below):
    print('Remove sales values above ' + str(upper) + ' and below ' + str(below))
    train = train[train['Sales'] < upper]  # remove very large sales values
    train = train[train['Sales'] >= below]  # remove negative large sales values
    return train
def interesting_days(df):
    # Since most of people in Russia are Orthodoxian, they do not celebrate christmas before start of january, but some study in 2015 
    # shows that Russia is starting to follow a more western trend when it comes to shopping (e.g., they shop a lot in mid-late december)
    # Therefore we chooses week 51-53. 
    df['special_days'] = df['week'].between(51, 53).astype(int) # christmas
    # The summer period (as we understood it) is also longer, but based on weather and when people usually have their vacation and stuff
    # we narrowed it down to week 26-31
    df['special_days'] += df['week'].between(26, 31).astype(int) # summer
    df['special_days'] += (df['week'] == 44).astype(int)
    df['special_daysPromo'] = df['special_days'].astype(str) + df['isPromo'].astype(str)

    # There seems to be more sale on friday and saturday, so they might bring some insight.
    df['party_time'] = df['day_of_week'].between(4,5).astype(int)
    df['party_timePromo'] = df['party_time'].astype(str) + df['isPromo'].astype(str)
    
    # School holidays in Russia 
    df['School_holiday'] = df['Date'].between('2015-01-01', '2015-01-15').astype(int)
    df['School_holiday'] += df['Date'].between('2015-03-23', '2015-03-27').astype(int)
    df['School_holiday'] += df['Date'].between('2015-05-26', '2015-08-31').astype(int)
    df['School_holiday'] += df['Date'].between('2015-11-02', '2015-11-06').astype(int)
    df['School_holiday'] += df['Date'].between('2016-01-01', '2016-01-15').astype(int)
    df['School_holiday'] += df['Date'].between('2016-03-21', '2016-03-25').astype(int)
    df['School_holiday'] += df['Date'].between('2016-05-30', '2016-08-31').astype(int)
    df['School_holiday'] += df['Date'].between('2016-11-07', '2016-11-11').astype(int)
    df['School_holiday'] += df['Date'].between('2017-01-02', '2017-01-15').astype(int)
    df['School_holiday'] += df['Date'].between('2017-03-22', '2017-03-29').astype(int)
    df['School_holiday'] += df['Date'].between('2017-06-01', '2017-09-01').astype(int)
    df['School_holiday'] += df['Date'].between('2017-11-05', '2017-11-12').astype(int)
    df['School_holidayPromo'] = df['School_holiday'].astype(str) + df['isPromo'].astype(str)
    return df
def promo_relation(df):
    # Add information about how long time ago there was promo and how long until next
    def days_to_promo(df):
        df["t"] = (df["Date"].where(df["isPromo"] > 0.0).bfill() - df["Date"]).dt.days
        return df
    def days_since_promo(df):
        df["t"] =  (df["Date"] - df["Date"].where(df["isPromo"] > 0.0).ffill()).dt.days
        return df
    
    df["days_to_promo"] = df.groupby([df.ts_id])[["Date", "isPromo", "SKU", "Store"]].apply(days_to_promo)["t"]
    df["days_since_promo"] = df.groupby([df.ts_id])[["Date", "isPromo", "SKU", "Store"]].apply(days_since_promo)["t"]
    return df

In [3]:
def model1_dataset(model3 = False, model4 = False):
    # Since model1, 3 and 4 are quite similar, we can use a single function to construct the dataset
    train = pd.read_csv('data/beer_train.csv',parse_dates=['Date'])
    test = pd.read_csv('data/beer_test.csv',parse_dates=['Date'])
    sku_features = pd.read_csv('data/sku_features.csv')
    id_map = pd.read_csv('data/id_store_sku.csv')
    
    train, test = create_lag(train, test)
    train = merge(train, sku_features, id_map)
    test = merge(test, sku_features, id_map)
    train = time_features(train)
    test = time_features(test)
    train = promo_features(train)
    test = promo_features(test)
    if model3 or model4:
        train = promo_relation(train)
        test = promo_relation(test)
        
    train = brand_fixer(train)
    test = brand_fixer(test)
    train, test = lager_fix(train, test)
    train = remove2015(train)
    
    if model4:
        remove = ['lag60Promo', 'lag180Promo', 'lag90Promo', 'lag365Promo', 'Brand']
        for i in remove:
            train.pop(i)
            test.pop(i)

    train = train.replace(to_replace='nannan', value=np.nan)
    test = test.replace(to_replace='nannan', value=np.nan)
    return train, test

def model2_dataset():
    train = pd.read_csv('data/beer_train.csv',parse_dates=['Date'])
    test = pd.read_csv('data/beer_test.csv',parse_dates=['Date'])
    sku_features = pd.read_csv('data/sku_features.csv')
    id_map = pd.read_csv('data/id_store_sku.csv')
    
    train, test = create_lag(train, test)
    train = merge(train, sku_features, id_map)
    test = merge(test, sku_features, id_map)
    train = time_features(train)
    test = time_features(test)
    train = promo_features(train)
    test = promo_features(test)

    train = interesting_days(train)
    test = interesting_days(test)
    train = remove_values(train, 15, 0)
    return train, test

def prepare_dataset(train, test):
    train_y = train['Sales']
    del(train['Sales'], train['Date'])
    train_features = train
    del(test['Date'], test['id'])
    test_features = test
    for c in test_features.columns:
        test_features[c] = test_features[c].astype('category')
        train_features[c] = train_features[c].astype('category')
    print('Number of features in train set:', len(train_features.columns))
    print('Number of features in test set:', len(test_features.columns))
    print('Features used in model:', train.columns)
    return train_y, train_features, test_features

In [4]:
train_model1, test_model1 = model1_dataset()
train_y1, train_features1, test_features1 = prepare_dataset(train_model1, test_model1)

train_model2, test_model2 = model2_dataset()
train_y2, train_features2, test_features2 = prepare_dataset(train_model2, test_model2)

# Model3 scores best
train_model3, test_model3 = model1_dataset(model3 = True)
train_y3, train_features3, test_features3 = prepare_dataset(train_model3, test_model3)

# Model4: Here we remove the lag promos and brand (as we got bad feature importance on them with RFE, RFECV, Shap)
train_model4, test_model4 = model1_dataset(model4 = True)
train_y4, train_features4, test_features4 = prepare_dataset(train_model4, test_model4)

Number of features in train set: 29
Number of features in test set: 29
Features used in model: Index(['ts_id', 'isPromo', 'lag_60', 'lag_90', 'lag_180', 'lag_365',
       'lag60Promo', 'lag180Promo', 'lag90Promo', 'lag365Promo', 'Store',
       'SKU', 'Segment', 'Pack', 'Product', 'Brand', 'Volume', 'day_of_month',
       'day_of_week', 'month', 'year', 'week', 'ts_promo', 'store_promo',
       'segment_promo', 'brand_promo', 'sku_promo', 'dom_promo', 'dow_promo'],
      dtype='object')
Remove sales values above 15 and below 0
Number of features in train set: 35
Number of features in test set: 35
Features used in model: Index(['ts_id', 'isPromo', 'lag_60', 'lag_90', 'lag_180', 'lag_365',
       'lag60Promo', 'lag180Promo', 'lag90Promo', 'lag365Promo', 'Store',
       'SKU', 'Segment', 'Pack', 'Product', 'Brand', 'Volume', 'day_of_month',
       'day_of_week', 'month', 'year', 'week', 'ts_promo', 'store_promo',
       'segment_promo', 'brand_promo', 'sku_promo', 'dom_promo', 'dow_promo'

## Final submission 1

This will produce the delivery file on Kaggle called FinalSubmission2.csv, which scored 0.45878 on the public leaderboard.

In [5]:
submission1 = pd.read_csv('data/intro_submission.csv')

lgb_param_model1 = {'n_estimators': 1566, 'learning_rate': 0.004370719636735594, 'num_leaves': 122,  
             'reg_alpha': 0, 'reg_lambda': 26, 'subsample': 0.9063104808260567, 'cat_l2': 17, 'cat_smooth': 10,
             'colsample_bytree': 0.22250521336587764, 'max_cat_threshold': 55, 'min_data_per_group': 70}

model_lgbm1 = lgbm.LGBMRegressor(**lgb_param_model1, random_state = 1)
model_lgbm1.fit(train_features1, train_y1)
prediction_model1 = model_lgbm1.predict(test_features1)

lgb_param_model2 = {'colsample_bytree': 0.1595013602757808, 'learning_rate': 0.004230626203615872, 'max_depth': 8, 
             'n_estimators': 1833, 'num_leaves': 118, 'reg_alpha': 15.418374764903756, 
             'reg_lambda': 32.11982305383127, 'subsample': 0.8928231814360663}

model_lgbm2 = lgbm.LGBMRegressor(**lgb_param_model2, random_state = 1)
model_lgbm2.fit(train_features2, train_y2)
prediction_model2 = model_lgbm2.predict(test_features2)

submission1['Sales'] = 0.65*prediction_model1 + 0.35*prediction_model2
submission1.to_csv('FinalSubmission2.csv', index=False)

## Final submission 2

This will produce the delivery file on Kaggle called FinalSubmission3.csv, which scored 0.45633 on the public leaderboard.

In [6]:
submission2 = pd.read_csv('data/intro_submission.csv')

lgb_param_model3 = {'n_estimators': 1915, 'learning_rate': 0.0049319743191803544, 'num_leaves': 116, 
                    'reg_alpha': 34, 'reg_lambda': 49, 'subsample': 0.9872621712765213, 'subsample_freq': 1, 
                    'colsample_bytree': 0.11944549125971712, 'max_cat_threshold': 60, 'min_data_per_group': 93, 
                    'cat_l2': 22, 'cat_smooth': 12}

submission3 = pd.read_csv('data/intro_submission.csv')
model_lgbm3 = lgbm.LGBMRegressor(**lgb_param_model3, random_state = 1)
model_lgbm3.fit(train_features3, train_y3)
prediction_model3 = model_lgbm3.predict(test_features3)

lgb_param_model4 = {'n_estimators': 2156, 'learning_rate': 0.004954170518408145, 'num_leaves': 108, 
             'reg_alpha': 40, 'reg_lambda': 21, 'subsample': 0.9261717756336443, 'subsample_freq': 1, 
             'colsample_bytree': 0.16903417694268677, 'max_cat_threshold': 63, 'min_data_per_group': 55, 'cat_l2': 23, 'cat_smooth': 15}
model_lgbm1 = lgbm.LGBMRegressor(**lgb_param_model4, random_state = 1)
model_lgbm1.fit(train_features4, train_y4)
prediction_model4 = model_lgbm1.predict(test_features4)

# Since model3 scores better, we put some more weight on it
submission2['Sales'] = 0.60*prediction_model3 + 0.40*prediction_model4
submission2.to_csv('FinalSubmission3.csv', index=False)